In [ ]:
import torch

In [ ]:
# load model
model_path = ''
model = torch.load(model_path, weights_only=False)

In [ ]:
from utils.dataloader import get_dataloader_and_tokenizer

_, tokenizer = get_dataloader_and_tokenizer(1)